### Training on POCOVID dataset

In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
import torch.nn as nn
from torchvision import transforms as tv_tsfm
from torch.utils.data import Dataset, DataLoader

# add models folder to path so that VGG16_model can be imported
import sys
sys.path.append('../models')
from vgg import VGG16_model

# add util folder to path so that PocovidDataset can be imported
sys.path.append('../util')
from pocovid_dataset import PocovidDataset

Use GPU if available

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


Instantiate the model with default parameters

In [3]:
model = VGG16_model()

Original Pocovidnet Model has the following trainable/nontrain parameters:

Total:         14,747,971

Trainable:      2,392,963

Non trainable: 12,355,008

In [4]:
total_params = sum(p.numel() for p in model.parameters())
print(f'{total_params:,} total parameters.')

total_trainable_params = sum(
    p.numel() for p in model.parameters() if p.requires_grad)
print(f'{total_trainable_params:,} training parameters.')

print (f'{total_params - total_trainable_params:,} non-trainable parameters.')

14,747,843 total parameters.
2,392,963 training parameters.
12,354,880 non-trainable parameters.


Define a loss function and an optimizer

In [5]:
criterion = nn.CrossEntropyLoss() #TODO change this to softmax()
optimizer = optim.Adam(params = model.parameters(), lr=0.001)

Load covid dataset using custom data loader

In [6]:
transformed_dataset = PocovidDataset(root_dir='../data/image_dataset',
                                     transform=tv_tsfm.Compose([
                                       tv_tsfm.Resize((224,224)),
                                       tv_tsfm.RandomAffine(10,translate=(0.1,0.1)),
                                       tv_tsfm.ToTensor()
                                       ]))

In [7]:
dataloader = DataLoader(transformed_dataset, batch_size=4, shuffle=True, num_workers=0)

In [8]:
#################################################
### TODO: data train/test/val split function? ###
#################################################
# return trainloader?


Train the model on covid dataset

In [9]:
for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(dataloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data[0].to(device), data[1].to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

KeyError: 0